In [1]:
import os
import pandas as pd
import glob
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
print(os.listdir("C:/Users/오원석/user_name/elo-merchant-category-recommendation"))

['Data_Dictionary.xlsx', 'historical_transactions.csv', 'merchants.csv', 'new_merchant_transactions.csv', 'sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
#conda install -c conda-forge workalendar

In [3]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings

import workalendar
from workalendar.america import Brazil

warnings.filterwarnings('ignore')
np.random.seed(4590)

In [60]:
cal = Brazil()
for yr in [2011,2012,2013,2014,2015,2016,2017]:
    print(yr,cal.holidays(yr))

2011 [(datetime.date(2011, 1, 1), 'New year'), (datetime.date(2011, 4, 21), "Tiradentes' Day"), (datetime.date(2011, 4, 24), 'Easter Sunday'), (datetime.date(2011, 5, 1), 'Labour Day'), (datetime.date(2011, 9, 7), 'Independence Day'), (datetime.date(2011, 10, 12), 'Our Lady of Aparecida'), (datetime.date(2011, 11, 2), "All Souls' Day"), (datetime.date(2011, 11, 15), 'Republic Day'), (datetime.date(2011, 11, 20), 'Consciência Negra'), (datetime.date(2011, 12, 25), 'Christmas Day')]
2012 [(datetime.date(2012, 1, 1), 'New year'), (datetime.date(2012, 4, 8), 'Easter Sunday'), (datetime.date(2012, 4, 21), "Tiradentes' Day"), (datetime.date(2012, 5, 1), 'Labour Day'), (datetime.date(2012, 9, 7), 'Independence Day'), (datetime.date(2012, 10, 12), 'Our Lady of Aparecida'), (datetime.date(2012, 11, 2), "All Souls' Day"), (datetime.date(2012, 11, 15), 'Republic Day'), (datetime.date(2012, 11, 20), 'Consciência Negra'), (datetime.date(2012, 12, 25), 'Christmas Day')]
2013 [(datetime.date(2013, 1,

In [4]:
train=pd.read_csv("C:/Users/오원석/user_name/elo-merchant-category-recommendation/train.csv")

In [28]:
history_trans = pd.read_csv("C:/Users/오원석/user_name/elo-merchant-category-recommendation/historical_transactions.csv")

In [6]:
merchant=pd.read_csv("C:/Users/오원석/user_name/elo-merchant-category-recommendation/merchants.csv")

In [7]:
new_merchant1=pd.read_csv("C:/Users/오원석/user_name/elo-merchant-category-recommendation/new_merchant_transactions.csv")

In [8]:
test=pd.read_csv("C:/Users/오원석/user_name/elo-merchant-category-recommendation/test.csv")

In [9]:
history_reader = pd.read_csv("C:/Users/오원석/user_name/elo-merchant-category-recommendation/historical_transactions.csv", chunksize = 100)
type(history_reader)

pandas.io.parsers.TextFileReader

In [10]:
hist_chunk = None

for chunk in history_reader:
    hist_chunk = chunk
    print(hist_chunk)
    break


   authorized_flag          card_id  city_id category_1  installments  \
0                Y  C_ID_4e6213e9bc       88          N             0   
1                Y  C_ID_4e6213e9bc       88          N             0   
2                Y  C_ID_4e6213e9bc       88          N             0   
3                Y  C_ID_4e6213e9bc       88          N             0   
4                Y  C_ID_4e6213e9bc       88          N             0   
5                Y  C_ID_4e6213e9bc      333          N             0   
6                Y  C_ID_4e6213e9bc       88          N             0   
7                Y  C_ID_4e6213e9bc        3          N             0   
8                Y  C_ID_4e6213e9bc       88          N             0   
9                Y  C_ID_4e6213e9bc       88          N             0   
10               Y  C_ID_4e6213e9bc       88          N             0   
11               Y  C_ID_4e6213e9bc       -1          N             0   
12               Y  C_ID_4e6213e9bc       88       

In [11]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2   
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64) 
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)   
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [12]:
df_train = reduce_mem_usage(train)

Mem. usage decreased to  4.04 Mb (56.2% reduction)


In [13]:

df_test = reduce_mem_usage(test)

Mem. usage decreased to  2.24 Mb (52.5% reduction)


In [29]:
hist_trans = reduce_mem_usage(history_trans)

Mem. usage decreased to 1749.11 Mb (43.7% reduction)


In [15]:
new_merchant = reduce_mem_usage(new_merchant1)

Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [30]:
for df in [hist_trans,new_merchant]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['date'] = df['purchase_date'].dt.date
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [31]:
hist_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,state_id,subsector_id,date,year,weekofyear,month,dayofweek,weekend,hour,month_diff
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,...,16,37,2017-06-25,2017,25,6,6,1,15,23
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,...,16,16,2017-07-15,2017,28,7,5,1,12,23
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,...,16,37,2017-08-09,2017,32,8,2,0,22,23
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,...,16,34,2017-09-02,2017,35,9,5,1,10,23
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,...,16,37,2017-03-10,2017,10,3,4,0,1,23


In [39]:
hist_trans['authorized_flag']=history_trans['authorized_flag'].map({'Y':1, 'N':0})
hist_trans['category_1']=history_trans['category_1'].map({'Y':1, 'N':0})

In [40]:
def aggregate_historical_transactions(history):
   
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
   
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'purchase_date': ['min','max'],
        'month_lag': ['min', 'max']
        }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['hist_' + '_'.join(col).strip()
                           for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
   
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='hist_transactions_count'))
   
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
   
    return agg_history

history = aggregate_historical_transactions(hist_trans)
del hist_trans
gc.collect()

1754

In [34]:
history.columns

Index(['card_id', 'hist_transactions_count', 'hist_authorized_flag_sum',
       'hist_authorized_flag_mean', 'hist_merchant_id_nunique',
       'hist_city_id_nunique', 'hist_purchase_amount_sum',
       'hist_purchase_amount_median', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_installments_sum', 'hist_installments_median',
       'hist_installments_max', 'hist_installments_min',
       'hist_installments_std', 'hist_purchase_date_min',
       'hist_purchase_date_max', 'hist_month_lag_min', 'hist_month_lag_max'],
      dtype='object')

In [37]:
hist_trans = reduce_mem_usage(history_trans)

Mem. usage decreased to 1693.59 Mb (50.4% reduction)


In [41]:
import datetime
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['year'] = df['first_active_month'].dt.year
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.date(2020, 1, 1) - df['first_active_month'].dt.date).dt.days
    df['working_time1'] = (df['first_active_month'].dt.date - datetime.date(2011, 1, 1)).dt.days
    df['working_time2'] = (df['first_active_month'].dt.date - datetime.date(2011, 12, 25)).dt.days
    df['working_time3'] = (df['first_active_month'].dt.date - datetime.date(2012, 1, 1)).dt.days
    df['working_time4'] = (df['first_active_month'].dt.date - datetime.date(2012, 12,25)).dt.days
    df['working_time5'] = (df['first_active_month'].dt.date - datetime.date(2013, 1, 1)).dt.days
    df['working_time6'] = (df['first_active_month'].dt.date - datetime.date(2013, 12, 25)).dt.days
    df['working_time7'] = (df['first_active_month'].dt.date - datetime.date(2014, 1, 1)).dt.days
    df['working_time8'] = (df['first_active_month'].dt.date - datetime.date(2014, 12, 25)).dt.days
    df['working_time9'] = (df['first_active_month'].dt.date - datetime.date(2015, 1, 1)).dt.days
    df['working_time10'] = (df['first_active_month'].dt.date - datetime.date(2015, 12, 25)).dt.days
    df['working_time11'] = (df['first_active_month'].dt.date - datetime.date(2016, 1, 1)).dt.days
    df['working_time12'] = (df['first_active_month'].dt.date - datetime.date(2016, 12, 25)).dt.days
    df['working_time13'] = (df['first_active_month'].dt.date - datetime.date(2017, 1, 1)).dt.days
    df['working_time14'] = (df['first_active_month'].dt.date - datetime.date(2017, 12, 25)).dt.days

In [42]:
new_merchant['authorized_flag']=new_merchant['authorized_flag'].map({'Y':1, 'N':0})
new_merchant['category_1']=new_merchant['category_1'].map({'Y':1, 'N':0})

In [43]:
new_merchant.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,state_id,subsector_id,date,year,weekofyear,month,dayofweek,weekend,hour,month_diff
0,1,C_ID_415bb3a509,107,0,1,B,307,M_ID_b0c793002c,1,-0.557617,...,9,19,2018-03-11,2018,10,3,6,1,14,23
1,1,C_ID_415bb3a509,140,0,1,B,307,M_ID_88920c89e8,1,-0.569336,...,9,19,2018-03-19,2018,12,3,0,0,18,23
2,1,C_ID_415bb3a509,330,0,1,B,507,M_ID_ad5237ef6b,2,-0.551270,...,9,14,2018-04-26,2018,17,4,3,0,14,22
3,1,C_ID_415bb3a509,-1,1,1,B,661,M_ID_9e84cda3b1,1,-0.671875,...,-1,8,2018-03-07,2018,10,3,2,0,9,23
4,1,C_ID_ef55cf8d4b,-1,1,1,B,166,M_ID_3c86fa3831,1,-0.659668,...,-1,29,2018-03-22,2018,12,3,3,0,21,23


In [44]:
def aggregate_new_transactions(new_trans):   
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'city_id': ['nunique'],
        'purchase_amount': ['sum', 'median', 'max', 'min', 'std'],
        'installments': ['sum', 'median', 'max', 'min', 'std'],
        'month_lag': ['min', 'max']
        }
    agg_new_trans = new_trans.groupby(['card_id']).agg(agg_func)
    agg_new_trans.columns = ['new_' + '_'.join(col).strip()
                           for col in agg_new_trans.columns.values]
    agg_new_trans.reset_index(inplace=True)
   
    df = (new_trans.groupby('card_id')
          .size()
          .reset_index(name='new_transactions_count'))
   
    agg_new_trans = pd.merge(df, agg_new_trans, on='card_id', how='left')
   
    return agg_new_trans

In [45]:
new_trans = aggregate_new_transactions(new_merchant)

In [46]:
new_trans.head()

,card_id,new_transactions_count,new_authorized_flag_sum,new_authorized_flag_mean,new_merchant_id_nunique,new_city_id_nunique,new_purchase_amount_sum,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_median,new_installments_max,new_installments_min,new_installments_std,new_month_lag_min,new_month_lag_max
0,C_ID_00007093c1,2,2,1,2,2,-1.328125,-0.664062,-0.656738,-0.671875,0.010704,2,1.0,1,1,0.000000,2,2
1,C_ID_0001238066,26,26,1,25,8,-14.851562,-0.649414,-0.078308,-0.740723,0.173462,42,1.0,10,-1,2.079941,1,2
2,C_ID_0001506ef0,2,2,1,1,1,-1.447266,-0.723633,-0.715332,-0.731934,0.011742,0,0.0,0,0,0.000000,1,1
3,C_ID_0001793786,31,31,1,31,7,-0.229004,-0.372803,3.130859,-0.737793,0.947266,0,0.0,0,0,0.000000,1,2
4,C_ID_000183fdda,11,11,1,11,2,-6.589844,-0.665527,-0.107666,-0.732422,0.182861,16,1.0,4,-1,1.293340,1,2


In [47]:
new_trans.columns


Index(['card_id', 'new_transactions_count', 'new_authorized_flag_sum',
       'new_authorized_flag_mean', 'new_merchant_id_nunique',
       'new_city_id_nunique', 'new_purchase_amount_sum',
       'new_purchase_amount_median', 'new_purchase_amount_max',
       'new_purchase_amount_min', 'new_purchase_amount_std',
       'new_installments_sum', 'new_installments_median',
       'new_installments_max', 'new_installments_min', 'new_installments_std',
       'new_month_lag_min', 'new_month_lag_max'],
      dtype='object')

In [48]:
train=pd.merge(df_train, new_trans, on='card_id', how='left')
test = pd.merge(df_test, new_trans, on='card_id', how='left')

In [49]:
train=pd.merge(df_train, history, on='card_id', how='left')
test = pd.merge(df_test, history, on='card_id', how='left')

In [50]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [51]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn import preprocessing
from sklearn import utils
from lightgbm import LGBMRegressor

In [52]:
use_cols1 = [col for col in train.columns if col not in ['card_id', 'first_active_month']]
use_cols2 = [col for col in test.columns if col not in ['card_id', 'first_active_month']]
train = train[use_cols1]
test = test[use_cols2]

In [56]:
y     = train['target']
train = train.drop(['target'],axis=1)

In [53]:
nfolds = 10
folds = KFold(n_splits=5, shuffle=True, random_state=4590)

In [54]:
param = {'num_leaves': 129,
         'min_data_in_leaf': 148,
         'objective':'regression',
         'max_depth': 9,
         'learning_rate': 0.005,
         "min_child_samples": 24,
         "boosting": "gbdt",
         "feature_fraction": 0.7202,
         "bagging_freq": 1,
         "bagging_fraction": 0.8125 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.3468,
         "verbosity": -1}

train.head()

,feature_1,feature_2,feature_3,target,year,month,elapsed_time,working_time1,working_time2,working_time3,...,hist_purchase_amount_std,hist_installments_sum,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_date_min,hist_purchase_date_max,hist_month_lag_min,hist_month_lag_max
0,5,2,1,-0.820312,2017,6,944,2343,1985,1978,...,0.212139,4,0.0,1,0,0.123314,1.498573,1.519551,-8,0
1,4,1,0,0.392822,2017,1,1095,2192,1834,1827,...,0.384967,543,1.0,10,-1,1.510777,1.483720,1.517438,-12,0
2,2,2,0,0.687988,2016,8,1248,2039,1681,1674,...,0.087380,0,0.0,0,0,0.000000,1.484123,1.519758,-13,0
3,4,3,0,0.142456,2017,9,852,2435,2077,2070,...,0.261624,84,1.0,3,-1,0.588974,1.506443,1.519818,-5,0
4,1,3,0,-0.159790,2017,11,791,2496,2138,2131,...,1.352094,182,1.0,12,1,1.896862,1.510445,1.519850,-3,0


In [57]:
feature_importance_df = np.zeros((train.shape[1], nfolds))
mvalid = np.zeros(len(train))
mfull  = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, train.values)):
    print('----')
    print("fold n°{}".format(fold_))
   
    x0,y0 = train.iloc[trn_idx], y[trn_idx]
    x1,y1 = train.iloc[val_idx], y[val_idx]
   
    trn_data = lgb.Dataset(x0, label= y0); val_data = lgb.Dataset(x1, label= y1)
   
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data],
                    verbose_eval=500, early_stopping_rounds = 150)
    mvalid[val_idx] = clf.predict(x1, num_iteration=clf.best_iteration)
   
    feature_importance_df[:, fold_] = clf.feature_importance()
   
    mfull += clf.predict(test, num_iteration=clf.best_iteration) / folds.n_splits
   
np.sqrt(mean_squared_error(mvalid, y))

----
fold n°0
Training until validation scores don't improve for 150 rounds
[500]	training's rmse: 3.66407	valid_1's rmse: 3.83344
Early stopping, best iteration is:
[816]	training's rmse: 3.63483	valid_1's rmse: 3.83067
----
fold n°1
Training until validation scores don't improve for 150 rounds
[500]	training's rmse: 3.68861	valid_1's rmse: 3.72351
Early stopping, best iteration is:
[756]	training's rmse: 3.66346	valid_1's rmse: 3.72222
----
fold n°2
Training until validation scores don't improve for 150 rounds
[500]	training's rmse: 3.68583	valid_1's rmse: 3.73336
Early stopping, best iteration is:
[575]	training's rmse: 3.6774	valid_1's rmse: 3.73315
----
fold n°3
Training until validation scores don't improve for 150 rounds
[500]	training's rmse: 3.68004	valid_1's rmse: 3.75901
Early stopping, best iteration is:
[629]	training's rmse: 3.66684	valid_1's rmse: 3.75868
----
fold n°4
Training until validation scores don't improve for 150 rounds
[500]	training's rmse: 3.68609	valid_1's 

3.757710330619495

39